# Summary

Thanks for Dave to share the great PMC vectorization file: http://evexdb.org/pmresources/vec-space-models/

The vectorization part is developed from the tutorials by Francois Chollet. A deeper cnn is developed from his network.

https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html



In [1]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from gensim.models import KeyedVectors
import pickle
from sklearn.model_selection import GridSearchCV, train_test_split
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.optimizers import Adam



MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 200

/home/dc/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 1 Embedding

In [2]:
p_file = 'data_X'

with open(p_file, 'rb') as fin:
    data_X = pickle.load(fin)

In [3]:
p_file = 'RCT_labels'

with open(p_file, 'rb') as fin:
    labels = pickle.load(fin)
labels = to_categorical(np.asarray(labels))

In [4]:
p_file = 'Models/vectors'

with open(p_file, 'rb') as fin:
    data = pickle.load(fin)

In [5]:
p_file = 'Models/embedding_matrix'

with open(p_file, 'rb') as fin:
    embedding_matrix = pickle.load(fin)
    

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.25)

In [7]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(MAX_NUM_WORDS,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

# 2 1D CNN

In [8]:
def model():
    # train a 1D convnet with global maxpooling
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    X = Conv1D(150, 1, activation='relu')(embedded_sequences)
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Dropout(0.16)(X)                                 # dropout
    X = MaxPooling1D(1)(X)

    X = Conv1D(150, 3, activation='relu')(X)
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Dropout(0.16)(X)                                 # dropout
    X = MaxPooling1D(3)(X)
    
    X = Conv1D(150, 5, activation='relu')(X)
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Dropout(0.16)(X)                                 # dropout
    X = MaxPooling1D(5)(X)

    X = GlobalMaxPooling1D()(X)
    preds = Dense(2, activation='softmax')(X)

    model = Model(inputs = sequence_input, outputs = preds)

        
    return model

In [9]:
print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 2, activation='relu')(embedded_sequences)
x = MaxPooling1D(2)(x)
x = Conv1D(128, 3, activation='relu')(x)
x = MaxPooling1D(3)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 8, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
preds = Dense(2, activation='softmax')(x)

model = Model(inputs = sequence_input, outputs = preds)


Training model.


In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 200)         4000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 999, 128)          51328     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 499, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 497, 128)          49280     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 165, 128)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 161, 128)          82048     
__________

In [11]:
from keras.utils import plot_model
plot_model(model, to_file='1dCNN.png')

<img src='1dCNN.png'>

In [12]:
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(X_train, Y_train,
          batch_size=128,
          epochs=20,
          validation_data=(X_test, Y_test))

Train on 14333 samples, validate on 4778 samples
Epoch 1/20
14333/14333 [==============================] - 8s 563us/step - loss: 0.4850 - acc: 0.7881 - val_loss: 0.3978 - val_acc: 0.8395
Epoch 2/20
14333/14333 [==============================] - 6s 443us/step - loss: 0.3902 - acc: 0.8400 - val_loss: 0.4112 - val_acc: 0.8368
Epoch 3/20
14333/14333 [==============================] - 6s 451us/step - loss: 0.3747 - acc: 0.8464 - val_loss: 0.3810 - val_acc: 0.8447
Epoch 4/20
14333/14333 [==============================] - 6s 443us/step - loss: 0.3627 - acc: 0.8534 - val_loss: 0.3949 - val_acc: 0.8405
Epoch 5/20
14333/14333 [==============================] - 6s 442us/step - loss: 0.3555 - acc: 0.8547 - val_loss: 0.3862 - val_acc: 0.8399
Epoch 6/20
14333/14333 [==============================] - 6s 443us/step - loss: 0.3423 - acc: 0.8604 - val_loss: 0.3719 - val_acc: 0.8464
Epoch 7/20
14333/14333 [==============================] - 6s 443us/step - loss: 0.3355 - acc: 0.8623 - val_loss: 0.3982 - v

In [13]:
model_json = model.to_json()
with open ("1dCNNModel.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("1dCNNModel.h5")

In [14]:
loss, acc = model.evaluate(X_test, Y_test)
print("Validation set accuracy = ", acc)

4778/4778 [==============================] - 1s 179us/step
Validation set accuracy =  0.8302637086896629


In [15]:
from sklearn.metrics import classification_report

pred = model.predict(X_test, batch_size=32, verbose=1)
predicted = np.argmax(pred, axis=1)
report = classification_report(np.argmax(Y_test, axis=1), predicted)
print(report)

4778/4778 [==============================] - 1s 173us/step
             precision    recall  f1-score   support

          0       0.89      0.79      0.84      2661
          1       0.77      0.88      0.82      2117

avg / total       0.84      0.83      0.83      4778

